In [15]:
import contextily
import sklearn
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from owslib.wfs import WebFeatureService
import os

In [16]:
wfs_url="http://sit.comune.bolzano.it/geoserver/wfs?service=WFS"

In [3]:
wfs = WebFeatureService(url=wfs_url, version='2.0.0')

In [4]:
layers = list(wfs.contents)

In [9]:
incidenti = []
for layer in layers:
    if layer.find('ciden') > -1:
        incidenti.append(layer)

In [17]:
for incidente in incidenti:
    response = wfs.getfeature(typename=incidente)
    data = gpd.read_file(response)
    data.crs="epsg:25832"
    data = data.to_crs(epsg=4326)
    data['DTINCID']=data['DTINCID'].astype(str)
    data.DTINCID =data.DTINCID.apply(lambda x: x.replace(' 00:00:00+00:00',""))
    data['lon'] = data.geometry.x
    data['lat'] = data.geometry.y
    name = incidente.replace("Cartografia:view_","")
    data.to_file("data" + os.sep + name + ".geojson",driver="GeoJSON")
    del data['geometry']
    data.to_csv("data" + os.sep + ".csv",index=False)

DriverError: Failed to create GeoJSON datasource: data/incidenti.geojson: data/incidenti.geojson: No such file or directory